# GANによる画像生成

commit用

```mycode/5_gan_generation/```

In [34]:
!gpustat

/bin/sh: gpustat: command not found


In [33]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms

## 5-3 Self-Attention GANの概要

2018年時点で最高峰のGANの１つである**BigGAN**もSAGANをベースとしているGANである．本節では

- Self-Attention
- pointwise convolution
- Spectral Normalization

の３つのテーマについての理解を目標とする．

### 従来のGANの問題点
従来のGANの転置畳み込みを繰り返す操作は特徴量マップが大きくなりますが，転置畳み込みの繰り返しでは**局所的な情報の拡大にしかならない**問題がある．

よってより良い画像生成を実現するためには可能であれば拡大する際に画像全体の大域的な情報を考慮する仕組みが必要になる．

### Self-Attentionの導入
従来のニューラルネットワークでは前の層の出力がそのまま次の層に入力されるので式としては
$$
y = x
$$
で表される．しかしこれでは局所的な性質が常に伝わっていくので，さらに大域的な情報を用いて表される$o$を導入する．この$o$に係数$\gamma$をかけた値を足して次の層に伝達することを考える．
$$
y = x +\gamma o
$$
この$o$をどのように求めるかについては以下の手順で求める．
1. $x$を[C,W,H]から[C,N]に変形する. $N=W\times H$
2. $S=x^Tx$ という行列を用意する．これは画像位置$i$と画像位置$j$の関係性を表す行列である．
3. $S$を行方向にソフトマックス関数にかける．これで画像位置$i$と画像位置$j$の関係性の総和が1になり扱いやすくなる．これを$\beta$とし，これを**Attention Map**という．
4. 上で作った$\beta$を$x$にかけることで$o$を得る．つまり$o = x\beta^T$である．$o=[C,N]$

この$o = x\beta^T$について
$$
o_{c=k,n=j} = \sum_{i=1}^Nx_{ki}\beta_{ji}
$$
この式はチャネル$k$において位置$j$について位置$i=1,2,\cdots N$の全ての影響を総和することを意味している．
$x_{ki}$ はチャネル$k$の位置$i$の特徴量を表していて，$\beta_{ji}$は位置$j$と位置$i$の関係性を表して，この$\sum$はそれらの積の総和を求めている．


In [39]:
#　最初のサイズ変形 (B,C,W,H)  to (B,C,N)
X = torch.randn(3,32,64,64)
X = X.view(X.shape[0], X.shape[1], -1)

# 掛け算
X_T = X.permute(0,2,1)
S = torch.bmm(X_T, X)

# 規格化
m = nn.Softmax(dim=-2)
attention_map_T = m(S)
attention_map = attention_map_T.permute(0,2,1)

# Self-Attention Map
o = torch.bmm(X, attention_map.permute(0,2,1))

### 1×1 Convolutions (pointwise convolution)
Self-Attention の制限は非常に強いのでこのままでは学習はうまくいかないことが多い．
そこでpointwise convolutionと呼ばれる前処理をした特徴量マップをSelf-Attentionに渡してあげることによって学習がうまくいくようにする．
またこれをすることによって特徴量マップのチャネル数を変えることができる(一般にはチャネル数を減らす)ので計算量を調整することができる．

またpointwise convolutionでは次の言葉が使われる
- query：元の入力$x$の転置に対応するもの
- key：元の入力$x$に対応するもの
- value：Attention Mapと掛け算する対象

出力のチャネルはカーネルの数となる．カーネルのチャネルは入力のチャネルに合わせる

In [47]:
X = torch.randn(3,32,64,64)

# 1×1の畳み込み層によるpointwise convolutionを用意　
query_conv = nn.Conv2d(
    in_channels=X.shape[1], out_channels=X.shape[1]//8, kernel_size=1)

key_conv = nn.Conv2d(
    in_channels=X.shape[1], out_channels=X.shape[1]//8, kernel_size=1)

value_conv = nn.Conv2d(
    in_channels=X.shape[1], out_channels=X.shape[1], kernel_size=1)

# 畳み込みをしてからサイズを変更する
proj_query = query_conv(X).view(
    X.shape[0], -1, X.shape[2]*X.shape[3])
proj_query = proj_query.permute(0,2,1)
proj_key = key_conv(X).view(
    X.shape[0], -1, X.shape[2]*X.shape[3])

# 掛け算
S = torch.bmm(proj_query, proj_key) # bmmはバッチごとの掛け算を行う

# 規格化
m = nn.Softmax(dim=-2)
attention_map_T = m(S)
attention_map = attention_map_T.permute(0,2,1)

# Self-Attention Map
proj_value = value_conv(X).view(
    X.shape[0], -1, X.shape[2]*X.shape[3])
o = torch.bmm(proj_value, attention_map.permute(0,2,1))

### Spectral Normalization
- Spectral Normalization：畳み込みの重みに対する規格化
- Batch Normalization：ディープラーニングモデルを流れるデータに対する規格化

***リプシッツ連続性(Lipschitz continuity)：***判別機Dの出力が入力データに対して頑健性を持つこと
具体的には層を表す行列の固有値のうち最大の固有値で全ての重みを割ることで規格化している．固有値は固有ベクトルの拡大率なので，一番拡大される固有値で全てを規格化することによって流れるデータが想定外に大きくなることを防ぐ．

In [50]:
z_dim = 20
image_size = 64
nn.utils.spectral_norm(nn.ConvTranspose2d(
    z_dim, image_size*8, kernel_size=4,stride=1))

ConvTranspose2d(20, 512, kernel_size=(4, 4), stride=(1, 1))